### 01: Preparation Phase (Package Loading)

In [1]:
from typing import List
import gurobipy as gp
from gurobipy import GRB

# Initialize Gurobi model
model = gp.Model("Gameplay_Optimization")

Restricted license - for non-production use only - expires 2025-11-24


### 02: Python Implementation Phase

In [16]:
class Keywords:
    """Manages Hearthstone keywords, focusing on Paladin, Warrior, Demon Hunter, and Hunter"""

    # General keywords available to all classes in this context
    GENERAL_KEYWORDS = {
        "Taunt", "Charge", "Rush",
        "Battlecry", "Deathrattle",
        "Windfury", "Lifesteal", "Poisonous", "Stealth",
        "Divine Shield"
    }

    # Class-specific keywords for Paladin, Warrior, Demon Hunter, and Hunter
    CLASS_KEYWORDS = {
        "Paladin": {"Divine Shield", "Taunt"},
        "Warrior": {"Taunt", "Rush"},
        "Demon Hunter": {"Rush", "Lifesteal"},
        "Hunter": {"Deathrattle", "Stealth"}
    }

    def __init__(self, keywords: List[str] = None, minion_class: str = "Neutral"):
        # Determine allowed keywords based on class and general keywords
        allowed_keywords = self.GENERAL_KEYWORDS.union(self.CLASS_KEYWORDS.get(minion_class, set()))
        # Initialize keywords with only valid ones for the given class
        self.keywords = {kw for kw in keywords if kw in allowed_keywords} if keywords else set()

    def add_keyword(self, keyword: str, minion_class: str = "Neutral"):
        """Adds a keyword if it's valid for the minion's class"""
        allowed_keywords = self.GENERAL_KEYWORDS.union(self.CLASS_KEYWORDS.get(minion_class, set()))
        if keyword in allowed_keywords:
            self.keywords.add(keyword)
        else:
            print(f"{keyword} is not allowed for {minion_class} minions.")

    def remove_keyword(self, keyword: str):
        """Removes a keyword if it exists"""
        if keyword in self.keywords:
            self.keywords.remove(keyword)

    def has_keyword(self, keyword: str) -> bool:
        """Checks if a specific keyword exists"""
        return keyword in self.keywords

    def __str__(self):
        return ", ".join(self.keywords)

In [18]:
class Minion:
    """Minion class, used to hold the keywords and properties of a minion"""

    def __init__(self, name: str, minion_class: str = "Neutral", keywords: List[str] = None, attack: int = 0, health: int = 0, strat_value: int = 0, mana_cost: int = 0):
        self.name = name
        self.minion_class = minion_class
        self.attack = attack
        self.health = health
        self.strat_value = strat_value
        self.mana_cost = mana_cost
        # Initialize keywords based on the minion's class
        self.keywords = Keywords(keywords, minion_class)

    def __str__(self):
        """Returns a description of the minion"""
        keywords = str(self.keywords) if self.keywords else "No Keywords"
        return f"Minion: {self.name} | Class: {self.minion_class} ttack:| A {self.attack} | Health: {self.health} | Keywords: {keywords}"

### Example Input and Attribute Extraction Phase

In [20]:
# Example usage: Define friendly and enemy minions
friendly_minions = [
    Minion(name="River Crocolisk", minion_class="Neutral", keywords=[], attack=2, health=3, strat_value=1, mana_cost=2),  # Basic minion
    Minion(name="Silverback Patriarch", minion_class="Neutral", keywords=["Taunt"], attack=1, health=4, strat_value=1, mana_cost=3),  # Basic taunt minion
    Minion(name="Chillwind Yeti", minion_class="Neutral", keywords=[], attack=4, health=5, strat_value=1, mana_cost=4),  # Strong neutral minion
    Minion(name="Kor'kron Elite", minion_class="Warrior", keywords=["Charge"], attack=4, health=3, strat_value=1, mana_cost=4),  # Charge minion
]

enemy_minions = [
    Minion(name="Silver Hand Recruit", minion_class="Paladin", keywords=[], attack=1, health=1, strat_value=1, mana_cost=1),  # Basic Paladin token
    Minion(name="Righteous Protector", minion_class="Paladin", keywords=["Taunt", "Divine Shield"], attack=1, health=1, strat_value=1, mana_cost=1),  # Basic Paladin minion
    Minion(name="Oasis Snapjaw", minion_class="Neutral", keywords=[], attack=2, health=7, strat_value=1, mana_cost=4),  # Neutral minion
]

hand = [
    Minion(name="Murloc Raider", minion_class="Neutral", keywords=[], attack=2, health=1, strat_value=1, mana_cost=1),  # Simple minion
    Minion(name="Bloodfen Raptor", minion_class="Neutral", keywords=[], attack=3, health=2, strat_value=1, mana_cost=2),  # Basic beast minion
    Minion(name="Kor'kron Elite", minion_class="Warrior", keywords=["Charge"], attack=4, health=3, strat_value=2, mana_cost=4),  # Another basic minion
]

In [24]:
# Number of minions
m = len(friendly_minions)  # Number of friendly minions
n = len(enemy_minions)     # Number of enemy minions
h = len(hand)  # Number of cards in hand 
M_t = 5  # Available mana for the current turn (e.g., turn 5)

# Combine friendly_minions + hand into a single list if you want to treat them similarly
combined_minions = friendly_minions + hand

A = [minion.attack for minion in combined_minions]
B = [minion.health for minion in combined_minions]
P = [minion.attack for minion in enemy_minions]     # Enemy minion attack values
Q = [minion.health for minion in enemy_minions]     # Enemy minion health values

# Strategic values and mana costs
S = [minion.strat_value for minion in hand]
C = [minion.mana_cost for minion in hand]

# Weights (example values, you can modify as needed)
W_1 = 1
W_2 = 1
W_3 = 1
W_4 = 1
W_5 = 1
W_6 = 1

### Gurobi Modeling Phase

In [26]:
import gurobipy as gp
from gurobipy import GRB

model = gp.Model("NewOptimizationModel")

# Decision variables
x_hero = model.addVars(m+h, vtype=GRB.BINARY, name="x_hero")  # Whether minion i attacks hero
x = model.addVars(m+h, n, vtype=GRB.BINARY, name="x")         # Whether minion i attacks enemy minion j
y = model.addVars(m+h, vtype=GRB.BINARY, name="y")            # Whether friendly minion i survives
z = model.addVars(n, vtype=GRB.BINARY, name="z")              # Whether enemy minion j survives
u = model.addVars(h, vtype=GRB.BINARY, name="u")              # Whether card k is played (k in 0..h-1)

# Objective function
objective = (
    W_1 * gp.quicksum(x_hero[i] * A[i] for i in range(m+h))
    - W_2 * gp.quicksum(z[j] * P[j] for j in range(n))
    + W_3 * gp.quicksum(y[i] * B[i] for i in range(m+h))
    + W_4 * gp.quicksum(A[i] * x[i, j] for i in range(m+h) for j in range(n))
    - W_5 * gp.quicksum(P[j] * x[i, j] for i in range(m+h) for j in range(n))
    + W_6 * gp.quicksum(u[k] * S[k] for k in range(h))  # Use S[k] for the k-th card in hand
)

model.setObjective(objective, GRB.MAXIMIZE)

# Hero health constraint
D_hero = gp.quicksum(x_hero[i] * A[i] for i in range(m))
H_hero = 30 - D_hero
model.addConstr(H_hero >= 0, "HeroHealth")

# Friendly minion attack constraint: each on-board minion can attack at most once
for i in range(m):
    model.addConstr(
        gp.quicksum(x[i, j] for j in range(n)) + x_hero[i] <= 1,
        f"AttackConstraint_{i}"
    )

# Friendly minion survival constraint
for i in range(m):
    for j in range(n):
        model.addConstr(
            y[i] <= 1 - ((P[j] - B[i] + 1) / max(P[j], 1)) * x[i, j],
            f"FriendlyMinionSurvival_{i}_{j}"
        )

# Enemy minion survival constraint
for j in range(n):
    model.addConstr(
        z[j] >= (Q[j] - gp.quicksum(x[i, j] * A[i] for i in range(m+h))) / Q[j],
        f"EnemyMinionSurvival_{j}"
    )

# Mana constraint
model.addConstr(
    gp.quicksum(u[k] * C[k] for k in range(h)) <= M_t,
    "ManaConstraint"
)

# Strategic value constraint
model.addConstr(
    gp.quicksum(S[k] * u[k] for k in range(h)) >= 0,
    "StrategicValueConstraint"
)

# Link in-hand minions' attacks to whether they are played
for i in range(m, m+h):
    # If the minion is not played, it cannot attack
    model.addConstr(x_hero[i] <= u[i-m], f"HandMinionAttackHero_{i}")
    for j in range(n):
        model.addConstr(x[i, j] <= u[i-m], f"HandMinionAttackMinion_{i}_{j}")

# Charge/Rush logic
for i in range(m, m+h):
    minion = hand[i-m]
    has_charge = minion.keywords.has_keyword("Charge")
    has_rush = minion.keywords.has_keyword("Rush")

    if not has_charge and not has_rush:
        # No Charge/Rush: can't attack at all this turn
        model.addConstr(x_hero[i] == 0, f"NoChargeRushHero_{i}")
        for j in range(n):
            model.addConstr(x[i, j] == 0, f"NoChargeRushMinion_{i}_{j}")
    elif has_rush and not has_charge:
        # Rush: can attack minions but not hero
        model.addConstr(x_hero[i] == 0, f"RushNoHero_{i}")
        # No further constraints needed since we already have u[i-m] controlling play
    elif has_charge:
        # Charge: can attack hero or minions if played
        # No additional constraints needed
        pass

# Taunt logic
tt = [1 if enemy_minion.keywords.has_keyword("Taunt") else 0 for enemy_minion in enemy_minions]
taunt_present = model.addVar(vtype=GRB.BINARY, name="taunt_present")

# If any taunt minion is alive, taunt_present = 1
for j in range(n):
    model.addConstr(taunt_present >= tt[j] * z[j], f"TauntPresentLower_{j}")

model.addConstr(
    taunt_present <= gp.quicksum(tt[j] * z[j] for j in range(n)),
    "TauntPresentUpper"
)

# If Taunt is present, no attacks on the hero
for i in range(m+h):
    model.addConstr(
        x_hero[i] <= 1 - taunt_present,
        f"RestrictHeroAttack_{i}"
    )

# Prioritize attacking Taunt minions if they exist
for i in range(m+h):
    model.addConstr(
        gp.quicksum(x[i, j] * (1 - tt[j]) for j in range(n))
        <= gp.quicksum(x[i, j] * tt[j] for j in range(n)) + (1 - taunt_present),
        f"PrioritizeTaunt_{i}"
    )

# Divine Shield handling
ds_active = model.addVars(n, vtype=GRB.BINARY, name="ds_active")

for j, enemy_minion in enumerate(enemy_minions):
    if enemy_minion.keywords.has_keyword("Divine Shield"):
        model.addConstr(
            ds_active[j] + gp.quicksum(x[i, j] for i in range(m+h)) <= 1,
            f"DivineShieldBreak_{j}"
        )
    else:
        model.addConstr(ds_active[j] == 0, f"NoDivineShield_{j}")

# Solve the model
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    print("Optimal objective value:", model.objVal)
    print("Enemy minions survival (z):")
    for j in range(n):
        print(f"z[{j}] = {z[j].X}")

    print("Friendly minions attacking enemy minions (x):")
    for i in range(m+h):
        for j in range(n):
            print(f"x[{i},{j}] = {x[i, j].X}")

    print("Friendly minions survival (y):")
    for i in range(m+h):
        print(f"y[{i}] = {y[i].X}")

    print("Cards played (u):")
    for k in range(h):
        print(f"u[{k}] = {u[k].X}")

    print("Taunt present:", taunt_present.X)
    for j in range(n):
        print(f"ds_active[{j}] = {ds_active[j].X}")
else:
    if model.status == GRB.INFEASIBLE:
        print("Model is infeasible.")
    elif model.status == GRB.UNBOUNDED:
        print("Model is unbounded.")
    else:
        print("Optimization was stopped with status", model.status)


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.2.0 24C100)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 63 rows, 45 columns and 162 nonzeros
Model fingerprint: 0xb35fc490
Variable types: 0 continuous, 45 integer (45 binary)
Coefficient statistics:
  Matrix range     [1e-01, 4e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 30.0000000
Presolve removed 63 rows and 45 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 10 available processors)

Solution count 2: 45 30 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.500000000000e+01, best bound 4.500000000000e+01, gap 0.0000%
Optimal objective value: 45.0
Enemy minions survival (z):
z[0] = 0.0
z[1] = 0.0
z[2] = 0.